In [23]:



import logging
import numpy as np

import matplotlib.pyplot as plt
from importlib import reload
import random

import tensorflow as tf
import importlib
import MCAgent as agent
import MCAgent3 as agent3
import Model as Con4
import All_In_One_environment as env
from tensorflow.python.client import device_lib
from tensorflow.keras.models import Sequential, load_model, Model
from tensorflow.keras.layers import Input, Dense, Conv2D, Flatten, BatchNormalization, Activation, LeakyReLU, add
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers

In [33]:
#local imports reload
importlib.reload(agent)
importlib.reload(agent3)
importlib.reload(Con4)
importlib.reload(env)

<module 'All_In_One_environment' from '/home/jevans40/AI/AIClass2021/Project/Group2-s_UW_Intro_AI_Project/Project_attempt_one/Project_attempt_one/All_In_One_environment.py'>

In [34]:
#settings:
#Settings
inputDim = (6,7,2)
outputDim = (7)
filterSize = 20
kernelDim = (4,4)
convLayers = 4
regConst = 0.0001
learningRate = 0.001
iteration = 1

Memory = []

In [35]:
def RunTwoAgents(Agent1,Agent2, Games, Verbosity = 0):
    memories = []
    for game in range(Games):
        print("Playing game {} out of {}".format(game,Games))
        winner = PlayGame(Agent1,Agent2,Verbosity)
        memories += Agent1.GetMemory()
        memories += Agent2.GetMemory()
        Agent1.ClearMemory()
        Agent2.ClearMemory()   

        tmp = Agent2
        Agent2 = Agent1
        Agent1 = tmp
    return memories
    #Return memory to train on

#Verbosity controls how much information to output
#0: None
#1: Winner/Loser
#2: Memory Output
def PlayGame(Agent1,Agent2,Verbosity=0):
    Agent1.SetPlayer(-1)
    Agent2.SetPlayer(1)
    done = False
    memories = []
    newGame = env.ConnectFour(1)
    num = random.randint(0,6)
    newGame.step(num,1)
    while not done:
        action = Agent1.GetAction(newGame)
        #Add some level of randomness so that we can get more varied cases
        nu = np.random.dirichlet([1]*7)/20
        action = action + nu
        newState,reward,done,info = newGame.step_array(action,Agent1.player)
        memories.append(action)
        if Verbosity > 1:
            print("Player -1 took the action\n{}".format(action))
        if done:
            if Verbosity > 0:
                print("Player -1 has won the game!")
            return -1

        action = Agent2.GetAction(newGame)
        nu = np.random.dirichlet([1]*7)/20
        action = action + nu
        newState,reward,done,info = newGame.step_array(action,Agent2.player)
        if Verbosity > 1:
            print("Player 1 took the action\n{}".format(action))
        if done:
            if Verbosity > 0:
                print("Player 1 has won the game!")
            return 1
        if Verbosity > 2:
            print(np.array(newGame.get_current_state()).reshape(6,7))
    
    #Return game result

def TrainModel(Memories, toFitAgent):
    for i in range(5):
        minbatch = random.sample(Memories,min(250,len(Memories)))

        trainingStates = np.array([Con4.ReshapeToModel(item[0],item[3]) for item in Memories])
        trainingTargets = np.array([item[1] for item in Memories])

        fit = toFitAgent.model.fit(trainingStates,trainingTargets,epochs=5,verbose=1,validation_split=0,batch_size = 32)

        toFitAgent.model.save("ModelCheckPointAgent3")
    return round(fit.history["loss"][0],4)

In [36]:
testModel = Con4.Agent3Model("./ModelCheckPointStart","./ModelCheckPointStartAgent1-2",inputDim,outputDim,filterSize,kernelDim,regConst,convLayers,learningRate)
testModel.InitModel()


In [37]:
Agent1 = agent3.ReinforcementAgent3(testModel,-1)
Agent2 = agent3.ReinforcementAgent3(testModel,1)

In [38]:
Mem = RunTwoAgents(Agent1, Agent2,5, Verbosity=2)
print(Mem)

Playing game 0 out of 5


AttributeError: 'NoneType' object has no attribute 'predict'

In [ ]:
TrainModel(Mem,testModel)

In [8]:
while True:
    Mem = RunTwoAgents(Agent1, Agent2,10, Verbosity=0)
    TrainModel(Mem,testModel)

Playing game 0 out of 10
delta
[-0.03412891 -0.03063705  0.04655539 -0.01891361  0.02456894 -0.03441732
  0.04716447]
delta
[ 4.85103731e-02 -3.28602392e-02  1.77000441e-02 -8.44769099e-05
  1.10934306e-02 -1.63765703e-02 -2.77930745e-02]
delta
[-0.01220193 -0.02975994  0.01270327 -0.00273657 -0.02305676  0.05715472
 -0.00193048]
delta
[-0.12939342 -0.02664055  0.13338247 -0.06341696  0.11640021  0.00687979
 -0.03693855]
delta
[-0.0165737  -0.00240327  0.02193083  0.00674381 -0.01696925 -0.01530297
  0.02273504]
delta
[-0.00860294  0.01447177  0.08479722  0.01481828 -0.04288911 -0.01880273
 -0.04359972]
delta
[-0.00133295  0.00378136 -0.02046532 -0.02250261  0.03361166 -0.02184831
  0.02892084]
delta
[-0.03589467 -0.02540567  0.00281058  0.04919889  0.01384981  0.00919509
 -0.01356819]
delta
[-0.04577402  0.01626072 -0.10789808  0.04209711  0.06611189 -0.10513923
  0.13459378]
delta
[-0.00298802 -0.03526873  0.03773952 -0.03491875  0.01347292  0.0376115
 -0.01543795]
delta
[-0.03413441

2021-12-13 00:40:22.762332: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: ModelCheckPointStart/assets
Epoch 1/5
6/6 [==============================] - 0s 9ms/step - loss: 0.1075
Epoch 2/5
6/6 [==============================] - 0s 8ms/step - loss: 0.1067
Epoch 3/5
6/6 [==============================] - 0s 8ms/step - loss: 0.1036
Epoch 4/5
6/6 [==============================] - 0s 7ms/step - loss: 0.1001
Epoch 5/5
6/6 [==============================] - 0s 8ms/step - loss: 0.0957
INFO:tensorflow:Assets written to: ModelCheckPointStart/assets
Epoch 1/5
6/6 [==============================] - 0s 7ms/step - loss: 0.0930
Epoch 2/5
6/6 [==============================] - 0s 9ms/step - loss: 0.0895
Epoch 3/5
6/6 [==============================] - 0s 8ms/step - loss: 0.0889
Epoch 4/5
6/6 [==============================] - 0s 7ms/step - loss: 0.0869
Epoch 5/5
6/6 [==============================] - 0s 6ms/step - loss: 0.0872
INFO:tensorflow:Assets written to: ModelCheckPointStart/assets
Epoch 1/5
6/6 [==============================] - 0s

KeyboardInterrupt: 